In [1]:
import pandas as pd


df = pd.read_csv('breast-cancer-clean.csv')

df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# gaussian process regression training

In [2]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [3]:
x = df[sorted(df.columns)]
for column in x.columns:
    print(column)
x = x.drop('diagnosis', axis=1)
y = df['diagnosis']




trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.2)

area_mean
area_se
area_worst
compactness_mean
compactness_se
compactness_worst
concave points_mean
concave points_se
concave points_worst
concavity_mean
concavity_se
concavity_worst
diagnosis
fractal_dimension_mean
fractal_dimension_se
fractal_dimension_worst
perimeter_mean
perimeter_se
perimeter_worst
radius_mean
radius_se
radius_worst
smoothness_mean
smoothness_se
smoothness_worst
symmetry_mean
symmetry_se
symmetry_worst
texture_mean
texture_se
texture_worst


In [4]:
sc=StandardScaler()
scaler = sc.fit(trainX)
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)

In [5]:
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(trainX_scaled, trainY)
gaussian_process.kernel_

0.532**2 * RBF(length_scale=2.46)

In [6]:
from sklearn.metrics import mean_squared_error, accuracy_score  # Import accuracy_score only if it's for classification

# Make predictions
y_pred = gaussian_process.predict(testX_scaled)

# Check if the task is classification or regression
if len(set(testY)) <= 2:  # Assuming binary classification
    y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary class labels
    print('Accuracy: {:.2f}'.format(accuracy_score(testY, y_pred_classes)))
else:
    mse = mean_squared_error(testY, y_pred)
    print('Mean Squared Error: {:.2f}'.format(mse))


Accuracy: 0.95


In [7]:
import pickle

with open('breast_cancer_gaussian.pkl', 'wb') as file:
    pickle.dump(gaussian_process, file)


with open('breast-cancer-scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)